In [5]:
# Install libraries if not already installed
!pip install pandas scikit-learn


In [6]:
#importing libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [11]:
#loading dataset
from google.colab import files
uploaded = files.upload()

dataset = pd.read_csv('super_market_dataset.csv')
dataset.head()


Saving super_market_dataset.csv to super_market_dataset.csv


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2021-103800,03-01-2021,07-01-2021,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2021-112326,04-01-2021,08-01-2021,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2021-112326,04-01-2021,08-01-2021,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2021-112326,04-01-2021,08-01-2021,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2021-141817,05-01-2021,12-01-2021,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


In [15]:
#check the first few rows of the dataset
dataset.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2021-103800,03-01-2021,07-01-2021,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2021-112326,04-01-2021,08-01-2021,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2021-112326,04-01-2021,08-01-2021,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2021-112326,04-01-2021,08-01-2021,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2021-141817,05-01-2021,12-01-2021,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


In [16]:
#check basic information about the dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          10194 non-null  int64  
 1   Order ID        10194 non-null  object 
 2   Order Date      10194 non-null  object 
 3   Ship Date       10194 non-null  object 
 4   Ship Mode       10194 non-null  object 
 5   Customer ID     10194 non-null  object 
 6   Customer Name   10194 non-null  object 
 7   Segment         10194 non-null  object 
 8   Country/Region  10194 non-null  object 
 9   City            10194 non-null  object 
 10  State/Province  10194 non-null  object 
 11  Postal Code     10194 non-null  object 
 12  Region          10194 non-null  object 
 13  Product ID      10194 non-null  object 
 14  Category        10194 non-null  object 
 15  Sub-Category    10194 non-null  object 
 16  Product Name    10194 non-null  object 
 17  Sales           10194 non-null 

In [17]:
#check the column names
print("Column Names:", dataset.columns)

Column Names: Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country/Region', 'City',
       'State/Province', 'Postal Code', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit'],
      dtype='object')


In [19]:
#combine relevant columns to create a 'content' feature
dataset['content'] = dataset['Product Name'] + " " + dataset['Category'] + " " + dataset['Sub-Category']

In [20]:
#drop rows with missing content
dataset = dataset.dropna(subset=['content'])


In [21]:
#verify the new column
dataset[['Product Name', 'content']].head()

,Product Name,content
0,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...","Message Book, Wirebound, Four 5 1/2"" X 4"" Form..."
1,GBC Standard Plastic Binding Systems Combs,GBC Standard Plastic Binding Systems Combs Off...
2,Avery 508,Avery 508 Office Supplies Labels
3,SAFCO Boltless Steel Shelving,SAFCO Boltless Steel Shelving Office Supplies ...
4,Avery Hi-Liter EverBold Pen Style Fluorescent ...,Avery Hi-Liter EverBold Pen Style Fluorescent ...


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#vectorize the content column
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(dataset['content'])

#compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

#function to recommend similar products
def recommend_products(product_name, cosine_sim=cosine_sim):
    # Get the index of the given product
    try:
        idx = dataset[dataset['Product Name'] == product_name].index[0]
    except IndexError:
        return "Product not found. Please check the product name."


    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get indices of the top 5 similar products (excluding the input product)
    sim_scores = sim_scores[1:6]
    product_indices = [i[0] for i in sim_scores]

    #return the top 5 recommended products
    return dataset.iloc[product_indices][['Product Name', 'Category', 'Sub-Category']]


In [23]:
#test with an example product
example_product = dataset['Product Name'].iloc[0]  #use the first product in the dataset
print(f"Recommendations for: {example_product}")

recommendations = recommend_products(example_product)
print(recommendations)


Recommendations for: Message Book, Wirebound, Four 5 1/2" X 4" Forms/Pg., 200 Dupl. Sets/Book
                                           Product Name         Category  \
128   Message Book, Wirebound, Four 5 1/2" X 4" Form...  Office Supplies   
1027  Message Book, Wirebound, Four 5 1/2" X 4" Form...  Office Supplies   
1228  Message Book, Wirebound, Four 5 1/2" X 4" Form...  Office Supplies   
3880  Message Book, Wirebound, Four 5 1/2" X 4" Form...  Office Supplies   
8036  Message Book, Wirebound, Four 5 1/2" X 4" Form...  Office Supplies   

     Sub-Category  
128         Paper  
1027        Paper  
1228        Paper  
3880        Paper  
8036        Paper  
